In [46]:
import pandas as pd
import numpy as np
import hashlib

In [2]:
from IPython.display import display, HTML

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

---

In [4]:
teams = pd.read_csv("../data/NCAA/Teams.csv")
season = pd.read_csv("../data/NCAA/Seasons.csv")
seeds = pd.read_csv("../data/NCAA/NCAATourneySeeds.csv")
regular_results = pd.read_csv("../data/NCAA/RegularSeasonCompactResults.csv")
tourney_results = pd.read_csv("../data/NCAA/NCAATourneyCompactResults.csv")
submission = pd.read_csv("../data/NCAA/SampleSubmissionStage1.csv")

In [7]:
display(teams.head())
display(season.head())
#display(seeds.head())
display(regular_results.head())
#display(tourney_results.head())
display(submission.head())

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


,ID,Pred
0,2014_1107_1110,0.5
1,2014_1107_1112,0.5
2,2014_1107_1113,0.5
3,2014_1107_1124,0.5
4,2014_1107_1140,0.5


---

# Análise pré 2014

In [84]:
results_before_2014 = regular_results[regular_results['Season'] < 2014]

In [85]:
results_before_2014

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
129199,2013,131,1458,68,1231,56,N,0
129200,2013,132,1274,87,1314,77,N,0
129201,2013,132,1279,66,1196,63,N,0
129202,2013,132,1326,50,1458,43,N,0


In [86]:
results_before_2014["match_name"] = (results_before_2014[["WTeamID", "LTeamID"]].min(axis=1)).astype(str) + "_"  + (results_before_2014[["WTeamID", "LTeamID"]].max(axis=1)).astype(str)

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_13308\2080191550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_before_2014["match_name"] = (results_before_2014[["WTeamID", "LTeamID"]].min(axis=1)).astype(str) + "_"  + (results_before_2014[["WTeamID", "LTeamID"]].max(axis=1)).astype(str)


In [87]:
results_before_2014['delta_score'] = (results_before_2014[["WScore", "LScore"]].max(axis=1)) - (results_before_2014[["WScore", "LScore"]].min(axis=1))

C:\Users\Nicolas\AppData\Local\Temp\ipykernel_13308\1380238973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_before_2014['delta_score'] = (results_before_2014[["WScore", "LScore"]].max(axis=1)) - (results_before_2014[["WScore", "LScore"]].min(axis=1))


In [88]:
results_before_2014 = results_before_2014.drop(columns=['DayNum','WLoc','NumOT'])
results_before_2014

,Season,WTeamID,WScore,LTeamID,LScore,match_name,delta_score
0,1985,1228,81,1328,64,1228_1328,17
1,1985,1106,77,1354,70,1106_1354,7
2,1985,1112,63,1223,56,1112_1223,7
3,1985,1165,70,1432,54,1165_1432,16
4,1985,1192,86,1447,74,1192_1447,12
...,...,...,...,...,...,...,...
129199,2013,1458,68,1231,56,1231_1458,12
129200,2013,1274,87,1314,77,1274_1314,10
129201,2013,1279,66,1196,63,1196_1279,3
129202,2013,1326,50,1458,43,1326_1458,7


In [149]:
results_before_2014_normalized = results_before_2014.copy(deep=True)
results_before_2014_normalized['Season'] = (results_before_2014_normalized['Season']-results_before_2014_normalized['Season'].min())/(results_before_2014_normalized['Season'].max()-results_before_2014_normalized['Season'].min())
results_before_2014_normalized['delta_score'] = (results_before_2014_normalized['delta_score']-results_before_2014_normalized['delta_score'].min())/(results_before_2014_normalized['delta_score'].max()-results_before_2014_normalized['delta_score'].min())

In [152]:
results_before_2014_normalized['Season'] = results_before_2014_normalized['Season'].replace(0,0.01)
results_before_2014_normalized['delta_score'] = results_before_2014_normalized['delta_score'].replace(0,0.01)

In [153]:
results_before_2014_normalized.head()

,Season,WTeamID,WScore,LTeamID,LScore,match_name,delta_score
0,0.01,1228,81,1328,64,1228_1328,0.177778
1,0.01,1106,77,1354,70,1106_1354,0.066667
2,0.01,1112,63,1223,56,1112_1223,0.066667
3,0.01,1165,70,1432,54,1165_1432,0.166667
4,0.01,1192,86,1447,74,1192_1447,0.122222


## Elo system

In [96]:
teams["elo_2014"] = 0
teams

,TeamID,TeamName,FirstD1Season,LastD1Season,elo_2014
0,1101,Abilene Chr,2014,2018,0
1,1102,Air Force,1985,2018,0
2,1103,Akron,1985,2018,0
3,1104,Alabama,1985,2018,0
4,1105,Alabama A&M,2000,2018,0
...,...,...,...,...,...
359,1460,Wright St,1988,2018,0
360,1461,Wyoming,1985,2018,0
361,1462,Xavier,1985,2018,0
362,1463,Yale,1985,2018,0


In [171]:
teams_list = teams['TeamID'].to_list()
for team in teams_list:
    wins = results_before_2014_normalized.loc[results_before_2014_normalized.WTeamID == team,'WTeamID'].count()
    losses = results_before_2014_normalized.loc[results_before_2014_normalized.LTeamID == team,'LTeamID'].count()
    diff = wins - losses
    teams.loc[teams.TeamID == team, 'TeamID'] = diff*2

In [172]:
teams

,TeamID,TeamName,FirstD1Season,LastD1Season,elo_2014
0,0,Abilene Chr,2014,2018,0
1,0,Air Force,1985,2018,-3520
2,0,Akron,1985,2018,-220
3,0,Alabama,1985,2018,2860
4,0,Alabama A&M,2000,2018,0
...,...,...,...,...,...
359,0,Wright St,1988,2018,0
360,0,Wyoming,1985,2018,820
361,0,Xavier,1985,2018,3500
362,0,Yale,1985,2018,-880


---